In [ ]:
% pip install beautifulsoup4
% pip install lxml

In [1]:
from bs4 import BeautifulSoup
import requests
import re

url = "https://www.imensa.de/{city}/{mensa}/{day}.html"
non_digit = re.compile('[^0-9,.]')

In [2]:
cities = {
    "muenster": [
        "mensa-da-vinci", "bistro-denkpause", "bistro-friesenring",
        "bistro-katholische-hochschule",
        "bistro-durchblick", "bistro-frieden", "bistro-kabu", "bistro-oeconomicum", "hier-und-jetzt", "mensa-am-aasee",
        "mensa-am-ring", "mensa-bispinghof"
    ]
}
days = ["montag", "dienstag", "mittwoch", "donnerstag", "freitag"]

In [3]:
attribute_replacement = {
    "vegan": "Vegan",
    "vegetarisch": "Vegetarian",
    "schwein": "Pork",
    "rind": "Beef",
    "geflügel": "Poultry",

    "farbstoff": "Dyed",
    "konservierungsstoffe": "Preservatives",
    "antioxidationsmittel": "Antioxidants",
    "geschmacksverstärker": "Flavor enhancers",
    "geschwefelt": "sulphurated",
    "geschwärzt": "blackened",
    "gewachst": "waxed",
    "phosphat": "Phosphate",
    "süßungsmittel": "Sweeteners",
    "phenylalaninquelle": "Phenylalanine source",

    "gluten": "Gluten",
    "dinkel": "Spelt",
    "gerste": "Barley",
    "hafer": "Oats",
    "kamut": "Kamut",
    "roggen": "Rye",
    "weizen": "Wheat",

    "krebstiere": "Crustaceans",
    "ei": "Egg",
    "fisch": "Fish",
    "erdnüsse": "Peanuts",
    "soja": "Soy",
    "milch": "Milk",

    "schalenfrüchte": "Nuts",
    "mandeln": "Almonds",
    "haselnüsse": "Hazelnuts",
    "walnüsse": "Walnuts",
    "cashewkerne": "Cashews",
    "pecannüsse": "Pecans",
    "paranüsse": "Brazil nuts",
    "pistazien": "Pistachios",
    "macadamianüsse": "Macadamias",

    "sellerie": "Celery",
    "senf": "Mustard",
    "sesam": "Sesame",
    "lupinen": "Lupines",
    "weichtiere": "Molluscs",
    "schwefeldioxid": "Sulfur dioxide",
}

In [13]:
c = list(cities.keys())[0]
print(c, cities[c][0], days[0])
f = requests.get(url.format(city=c, mensa=cities[c][0], day=days[0]))
document = f.text

muenster mensa-da-vinci montag


In [10]:
soup = BeautifulSoup(document, "lxml")

In [11]:
mensa_rating_avg = float(soup.find(class_="aw-ratings-average").text.replace(",", "."))
mensa_rating_count = int(soup.find(class_="aw-ratings-count").text.replace(".", ""))

print(mensa_rating_count, mensa_rating_avg)

2001 3.8


In [12]:
main_meal = True
for div in soup.find_all(class_="aw-meal-category"):
    for meal in div.find_all(class_="aw-meal row no-margin-xs"):
        name = meal.find(class_="aw-meal-description").text
        price_for_students = meal.find(title="Preis für Studierende").text[:-2].replace(",", ".")
        price_for_non_students = price_for_students * 1.5

        attributes = meal.find(class_="small aw-meal-attributes").span.text.replace("\xa0", "").split(" ")
        category = attribute_replacement[attributes[0].lower()]

        att_type = 0
        additives = []
        allergies = []

        for att in attributes[1:]:
            att = att.lower()
            replacement = att
            if att in attribute_replacement:
                replacement = attribute_replacement[att]

            if att == "zusatz":
                att_type = 1
            elif att == "allergen":
                att_type = 2
            elif att_type == 1:
                additives.append(replacement)
            elif att_type == 2:
                allergies.append(replacement)

        last_occurrence = meal.find(title="Zuletzt angeboten")
        if last_occurrence:
            last_occurrence = last_occurrence.text

        ratings_count_tag = meal.find(class_="aw-meal-histogram-count")
        ratings_count = 0
        if ratings_count_tag:
            ratings_count = int(non_digit.sub("", ratings_count_tag.text))

        ratings_avg_tag = meal.find(class_="aw-meal-histogram-average")
        ratings_avg = 0
        if ratings_avg_tag:
            ratings_avg = float(non_digit.sub("", ratings_avg_tag.text.replace(",", ".")))

        print(name, (price_for_students, price_for_non_students), main_meal, category, additives, allergies,
              last_occurrence, ratings_count, ratings_avg)
    main_meal = False

Backkartoffel mit Zaziki 1.65 True Vegan [] ['Mustard', 'Soy'] None 0 0
Hähnchen Piccata mit Tomatensauce 3.10 True Poultry [] ['Gluten', 'Milk', 'Wheat'] 05.09.2022 4 4.3
Ravioli mit Ricotta-Spinatfüllung mit Kräutersauce 1.40 True Vegetarian [] ['Gluten', 'Milk', 'Wheat'] None 1 5.0
Prinzessbohnen 0.30 False Vegan ['Dyed'] [] 07.11.2022 14 4.4
Reis 0.30 False Vegan ['Dyed'] [] 09.11.2022 29 3.2
Kohlrabi 0.30 False Vegan ['Dyed'] [] 11.11.2022 5 4.6
Kaisergemüse 0.50 False Vegan ['Dyed'] [] 07.11.2022 9 4.2
Pommes Frites klein 0.70 False Vegan [] [] 07.11.2022 5 4.4
Beilagensalat mit Essig-Öl-Dressing 0.90 False Vegan [] ['Mustard'] 11.11.2022 9 3.2
Himbeerquark 0.70 False Vegetarian [] ['Milk'] 04.11.2022 2 1.0
Quark mit Blaubeeren 0.70 False Vegetarian [] ['Milk'] 10.11.2022 0 0
